In [ ]:
!pip install numpy pandas scikit-learn

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from math import sqrt

In [9]:
# 데이터 불러오기
train_data = pd.read_csv('./Data/train.csv')
test_data = pd.read_csv('./Data/test.csv')
original_test_data = test_data.copy()

In [ ]:
print(train_data)

In [ ]:
# 'Income'을 예측 변수로, 나머지를 특성으로 사용
X = train_data.drop(['ID', 'Income', 'Birth_Country', 'Birth_Country (Father)', 'Birth_Country (Mother)', 'Citizenship', 'Race', 'Losses', 'Citizenship', 'Household_Summary'], axis=1)
y = train_data['Income']

# 범주형 변수를 더미 변수로 변환
X = pd.get_dummies(X)
test_data = pd.get_dummies(test_data.drop(['ID'], axis=1))
test_data = test_data.reindex(columns=X.columns, fill_value=0)

In [10]:
print(test_data)

Stacked model with improvements predictions saved to stacked_submission_with_improvements.csv


In [ ]:
# 기본 모델 정의
base_models = [
    ('rf_model', RandomForestRegressor(random_state=42)),
    ('gb_model', GradientBoostingRegressor(random_state=42))
]

In [ ]:
# 스태킹을 위한 훈련 데이터 준비
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 기본 모델의 예측 저장을 위한 배열 초기화
train_pred = np.zeros((X_train.shape[0], len(base_models)))
val_pred = np.zeros((X_val.shape[0], len(base_models)))
test_pred = np.zeros((test_data.shape[0], len(base_models)))

In [ ]:
# 기본 모델 훈련 및 예측
for i, (name, model) in enumerate(base_models):
    print(f'Training {name}...')
    model.fit(X_train, y_train)
    train_pred[:, i] = model.predict(X_train)
    val_pred[:, i] = model.predict(X_val)
    test_pred[:, i] = model.predict(test_data)

# 메타 모델 훈련
meta_model = LinearRegression()
meta_model.fit(val_pred, y_val)

# 최종 예측
final_predictions = meta_model.predict(test_pred)

In [ ]:
# 예측 결과를 제출 형식에 맞게 저장
submission = pd.DataFrame({'ID': original_test_data['ID'], 'Income': final_predictions})
submission.to_csv('stacked_submission.csv', index=False)

print('Drop_submission.csv')